# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents
5. Multi-Agents

In [1]:
import json
import pandas as pd
import random
import uvicorn
from pathlib import Path
from datetime import datetime
from sklearn.metrics import accuracy_score
from pydantic import BaseModel, Field
from typing import Any
from fastapi import FastAPI
from io import StringIO
from language_models.agents.chain import AgentChain
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.tools.tool import Tool
from language_models.proxy_client import BTPProxyClient
from language_models.agents.react import ReActAgent
from language_models.tools.earthquake import earthquake_tools
from language_models.tools.current_date import current_date_tool
from language_models.settings import settings

In [2]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.0,
)

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Sentiment: Positive


In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

positive


In [6]:
system_prompt = "Take the following movie review and determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Wow! This movie was incredible. The acting was superb, and the plot kept me on the edge of my seat. I highly recommend it!"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

1


In [7]:
system_prompt = "Take the following movie review determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

I'm sorry, I am an AI language model and I do not have access to real-time weather information. I recommend checking a reliable weather website or using a weather app to get the most accurate and up-to-date forecast for Seattle.


In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review. 

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

-1


## 2. Tools

In [9]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12" # answer: $1,289.98

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Total Raw Cost = $1,290.98


In [10]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression.")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations.",
    args_schema=Calculator
)

print(calculator_tool)

tool name: Calculator, tool description: Use this tool when you want to do calculations., tool input: {{'expression': {{'description': 'A math expression.', 'title': 'Expression', 'type': 'string'}}}}


In [11]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Use a JSON blob to specify a thought, a tool by providing an tool key (tool name) and a tool_input key (tool input).

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "To calculate the total raw cost, you need to add up all the individual costs.",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


In [12]:
print(calculator(**response["tool_input"]))

1289.98


In [13]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Use a JSON blob to specify a thought, a tool by providing an tool key (tool name) and a tool_input key (tool input).

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}

Observation: tool result
... (this Thought/Tool/Observation can repeat N times)

When you know the answer, use the following JSON format:
{{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=json.dumps(response)),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=f"Response of Calculator tool: {calculator(**response['tool_input'])}"),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "I now know the total raw cost.",
    "tool": "Final Answer",
    "tool_input": "The total raw cost is $1289.98."
}


## 3. Chat-based AI Agents

### Earthquake

In [14]:
system_prompt = """You are an United States Geological Survey expert who can answer questions regarding earthquakes and can run forecasts.

Use the current date tool to access the local date and time before using other tools.

Take the following question and answer it as accurately as possible."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=2048,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

earthquake_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{question}",
    task_prompt_variables=["question"],
    tools=earthquake_tools + [current_date_tool],
    output_format=Output,
    iterations=10,
)

In [15]:
response = earthquake_agent.invoke({"question": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

11/05/24 11:59:31 INFO Prompt:
How many earthquakes have occurred for the past week with a magnitude of 5 or greater?
11/05/24 11:59:34 INFO Raw response:
{
  "thought": "First, I need to get the current date to calculate the start date for the past week.",
  "tool": "Current Date",
  "tool_input": {}
}
11/05/24 11:59:34 INFO Thought:
First, I need to get the current date to calculate the start date for the past week.
11/05/24 11:59:34 INFO Tool:
Current Date
11/05/24 11:59:34 INFO Tool input:
{}
11/05/24 11:59:34 INFO Tool response:
2024-05-11 11:59:34.368692
11/05/24 11:59:37 INFO Raw response:
{
  "thought": "I need to count the number of earthquakes that have occurred in the past week with a magnitude of 5 or greater.",
  "tool": "Count",
  "tool_input": {
    "start_time": "2024-05-04T11:59:34.368692",
    "min_magnitude": 5
  }
}
11/05/24 11:59:37 INFO Thought:
I need to count the number of earthquakes that have occurred in the past week with a magnitude of 5 or greater.
11/05/24

In [16]:
print(response.final_answer["content"])

There have been 30 earthquakes in the past week with a magnitude of 5 or greater.


In [17]:
response = earthquake_agent.invoke({"question": "Query 10 earthquakes that occurred yesterday and have a magnitude > 3."})

11/05/24 11:59:43 INFO Prompt:
Query 10 earthquakes that occurred yesterday and have a magnitude > 3.
11/05/24 11:59:45 INFO Raw response:
Tool response:
2024-05-12 11:59:34.368692
11/05/24 11:59:47 INFO Raw response:
Tool response:
2024-05-10 11:59:34.368692
11/05/24 11:59:52 INFO Raw response:
{
  "thought": "I need to get the current date to calculate the date for 'yesterday'.",
  "tool": "Current Date",
  "tool_input": {}
}
11/05/24 11:59:52 INFO Thought:
I need to get the current date to calculate the date for 'yesterday'.
11/05/24 11:59:52 INFO Tool:
Current Date
11/05/24 11:59:52 INFO Tool input:
{}
11/05/24 11:59:52 INFO Tool response:
2024-05-11 11:59:52.052926
11/05/24 11:59:57 INFO Raw response:
{
  "thought": "I need to query the earthquakes that occurred yesterday with a magnitude greater than 3.",
  "tool": "Query",
  "tool_input": {
    "start_time": "2024-05-10T00:00:00",
    "end_time": "2024-05-10T23:59:59",
    "min_magnitude": 3,
    "limit": 10
  }
}
11/05/24 11:59

In [18]:
print(response.final_answer["content"])

Here are 10 earthquakes that occurred yesterday with a magnitude greater than 3:

1. [M 4.2 - Rat Islands, Aleutian Islands, Alaska](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mxv1)
2. [M 4.2 - western Xizang](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mxui)
3. [M 3.8 - 54 km NW of Toyah, Texas](https://earthquake.usgs.gov/earthquakes/eventpage/tx2024jfca)
4. [M 4.9 - 91 km SSW of Kencong, Indonesia](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mxu2)
5. [M 4.7 - 106 km SE of Angoram, Papua New Guinea](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mxtw)
6. [M 3.1 - 7 km NW of Caban, Puerto Rico](https://earthquake.usgs.gov/earthquakes/eventpage/pr71448293)
7. [M 4.6 - 71 km WNW of El Aguilar, Argentina](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mxru)
8. [M 5.1 - 85 km WNW of Bengkulu, Indonesia](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mxrs)
9. [M 4.8 - 101 km SE of Pangai, Tonga](https://earthquake.usgs.gov/ear

In [19]:
response = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

11/05/24 12:00:28 INFO Prompt:
Can MegaQuakes really happen? Like a magnitude 10 or larger?
11/05/24 12:00:43 INFO Raw response:
{
  "thought": "A magnitude 10 or larger earthquake, often referred to as a 'megaquake', is theoretically possible but extremely unlikely. The magnitude of an earthquake is related to the length of the fault on which it occurs - the longer the fault, the larger the earthquake. The simple truth is that there are no known faults capable of generating a magnitude 10 or larger earthquake. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960, and it occurred on a fault that was over 1,000 miles long and 150 miles wide. To generate a magnitude 10 would require a fault that is several times larger, and there are no such faults on Earth.",
  "tool": "Final Answer",
  "tool_input": {"content": "A magnitude 10 or larger earthquake, often referred to as a 'megaquake', is theoretically possible but extremely unlikely. The magnitude of an earthquake i

In [20]:
print(response.final_answer["content"])

A magnitude 10 or larger earthquake, often referred to as a 'megaquake', is theoretically possible but extremely unlikely. The magnitude of an earthquake is related to the length of the fault on which it occurs - the longer the fault, the larger the earthquake. The simple truth is that there are no known faults capable of generating a magnitude 10 or larger earthquake. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960, and it occurred on a fault that was over 1,000 miles long and 150 miles wide. To generate a magnitude 10 would require a fault that is several times larger, and there are no such faults on Earth.


## 4. Service-based AI Agents

### Contract Drafting

In [21]:
system_prompt = """Take the follow content and generate a draft of a section for a contract.

Write the content from scratch and make it sound professional.

Respond with the title of the section and the content of the section."""

task_prompt = """Section content:
{section}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class ContractSection(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")

contract_drafting_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["section", "section_name"],
    tools=None,
    output_format=ContractSection,
    iterations=5,
)

In [22]:
def generate_contract(contract_sections: list[str]) -> str:
    sections = []
    for contract_section in contract_sections:
        response = contract_drafting_agent.invoke({"section": contract_section})
        section = response.final_answer["title"] + "\n\n" + response.final_answer["content"]
        sections.append(section)
    return "\n\n".join(sections)

In [23]:
definitions = "Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA."

amendment = """INVOICING AND PAYMENT TERMS
Clause 12.1(ii) of the GMA shall be cancelled and substituted as follow:
[*****]
[*****]
[*****]

Any other provision of Clause 12 shall remain in full force and effect.

PRICE CONDITIONS
(i) Clause 3.2 of the Exhibit 14 of the GMA shall be cancelled and substituted as follow:
“3.2 Technical conditions for prices adjustment
The prices set out in this Exhibit 14 shall be modified every [*****] at the occasion of the invoicing reconciliation pursuant to Clause 11
(“Reconciliation”) if the Standard Operations of the Aircraft, analyzed at the time of the adjustment (all calculations are made with figures corresponding to [*****], change by more or less
[*****] with respect to the estimated values of the same parameters, considered at the time of commencement of the Term.
As from the date this Agreement enters into force, the Parties agree to take into account the following basic operating parameters (the
“Standard Operations”) as a reference for the above calculation:
[*****]
[*****]
[*****]"""

effective_date_and_duration = "Amendment is effective starting on the date of its signature by both Parties."

confidentiality = """Confidential Information released by either of the Parties (the “Disclosing Party”) to the
other Party (the “Receiving Party”) shall not be released in whole or in part to any third party:
- Not to deliver, disclose or publish it to any third party including subsidiary companies and companies having an interest in its capital
- Use Confidential Information solely for the purpose of this Amendment
- Disclose the Confidential Information only to those of its direct employees
- Not to duplicate the Confidential Information nor to copy

Any Confidential Information shall remain the property of the Disclosing Party.

The Receiving Party hereby acknowledges and recognises that Confidential Information is protected by copyright Laws and related
international treaty provisions, as the case may be.

This shall survive termination or expiry of this Amendment for a period of five (5) years following such End Date."""

governing_law = """Pursuant to and in accordance with Section 5-1401 of the New York General Obligations Law.

Arbitration: in the event of a dispute arising out of or relating to this Amendment, including without limitation disputes regarding the
existence, validity or termination of this Amendment (a “Dispute”), either Party may notify such Dispute to the other through service of a
written notice (the “Notice of Dispute”). 

Arbitration, and any proceedings, and meetings incidental to or related to the arbitration process, shall take place in New York.

Arbitration shall be kept confidential and the existence of the proceeding and any element.

During any period of negotiation or arbitration, the Parties shall continue to meet their respective obligations.

Notwithstanding any provision of this the Parties may, at any time, seek and decide to settle a Dispute.

Judgment upon any award may be entered in any court having jurisdiction.

Recourse to jurisdictions is expressly excluded except as provided for in the ICC Rules of Conciliation and Arbitration."""

miscellaneous = """Amendment contains the entire agreement between the Parties regarding the subject-matter.

Amendment shall not be varied or modified except by a written document duly signed."""

contract = generate_contract(
    contract_sections=[
        definitions,
        amendment,
        effective_date_and_duration,
        confidentiality,
        governing_law,
        miscellaneous,
    ]
)

11/05/24 12:00:43 INFO Prompt:
Section content:
Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA.
11/05/24 12:00:50 INFO Raw response:
{
  "thought": "The user wants a professional sounding contract section that explains how capitalized terms, whether singular or plural, used in an amendment will have the same meaning as in the GMA (General Master Agreement). I will draft this section and title it 'Interpretation of Terms'.",
  "tool": "Final Answer",
  "tool_input": {
    "title": "Interpretation of Terms",
    "content": "All capitalized terms utilized within this Amendment, irrespective of their singular or plural form, shall retain the identical interpretation as defined within the General Master Agreement (GMA)."
  }
}
11/05/24 12:00:50 INFO Thought:
The user wants a professional sounding contract section that explains how capitalized terms, whether singular or plural, used in an amendment will have the same meaning as in the GM

In [24]:
print(contract)

Interpretation of Terms

All capitalized terms utilized within this Amendment, irrespective of their singular or plural form, shall retain the identical interpretation as defined within the General Master Agreement (GMA).

Interpretation of Terms and Amendment of Payment and Pricing Conditions

INTERPRETATION OF TERMS: Any terms capitalized in this Amendment, whether in singular or plural form, shall bear the same interpretation as defined in the General Master Agreement (GMA).

AMENDMENT OF INVOICING AND PAYMENT TERMS: The existing Clause 12.1(ii) of the GMA is hereby revoked and replaced with the following provisions: [*****]. All other provisions under Clause 12 shall continue to be in full force and effect.

AMENDMENT OF PRICE CONDITIONS: The existing Clause 3.2 of Exhibit 14 of the GMA is hereby revoked and replaced with the following provision: '3.2 Technical conditions for price adjustment: The prices outlined in this Exhibit 14 shall be adjusted every [*****] during the invoici

### Sentiment Analysis

In [25]:
df_tweets = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.where(df_tweets.sentiment != 2)
df_tweets["sentiment"] = df_tweets["sentiment"].map({4: 1, 0: 0})
df_tweets_sampled = df_tweets.sample(n=10)
df_tweets_sampled.head()

,sentiment,id,date,query,user,tweet
776854,0,2322260184,Wed Jun 24 22:18:43 PDT 2009,NO_QUERY,jannastewart,it's my last full day in rome. im going to mi...
1580299,1,2190047147,Tue Jun 16 01:37:01 PDT 2009,NO_QUERY,minnypink,Good morning hair - breakfast - doctor - shor...
380916,0,2052671027,Sat Jun 06 01:34:21 PDT 2009,NO_QUERY,nitnatoz,@Rove1974 Awwww crap! Thanks! Saturday afterno...
856266,1,1573718457,Tue Apr 21 02:31:24 PDT 2009,NO_QUERY,Moriasreadings,Morning everyone
1363173,1,2049462263,Fri Jun 05 16:55:26 PDT 2009,NO_QUERY,dsreflect,4 trips through krispy kreme 8D the guy gave u...


In [26]:
system_prompt = """Take the following tweet and determine the sentiment of the review. 

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class Sentiment(BaseModel):
    sentiment: int = Field(description="The sentiment of the tweet.")

sentiment_analysis_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Tweet:\n{tweet}",
    task_prompt_variables=["tweet"],
    tools=None,
    output_format=Sentiment,
    iterations=2,
)

In [27]:
def classify_sentiment(tweet: str) -> int:
    response = sentiment_analysis_agent.invoke({'tweet': tweet})
    return response.final_answer['sentiment'] or 0

In [28]:
df_tweets_sampled["prediction"] = [classify_sentiment(tweet) for tweet in df_tweets_sampled.tweet]

11/05/24 12:02:07 INFO Prompt:
Tweet:
it's my last full day in rome.  im going to miss this place.
11/05/24 12:02:17 INFO Raw response:
{
  "thought": "The tweet expresses a positive sentiment towards Rome, indicating that the user will miss the place.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 1}
}
11/05/24 12:02:17 INFO Thought:
The tweet expresses a positive sentiment towards Rome, indicating that the user will miss the place.
11/05/24 12:02:17 INFO Final answer:
{'sentiment': 1}
11/05/24 12:02:17 INFO Prompt:
Tweet:
Good morning  hair - breakfast - doctor - short shopping trip 
11/05/24 12:02:20 INFO Raw response:
{
  "thought": "The tweet seems to be neutral as it's just describing the user's plans for the day. There's no clear positive or negative sentiment.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 1}
}
11/05/24 12:02:20 INFO Thought:
The tweet seems to be neutral as it's just describing the user's plans for the day. There's no clear positive or ne

In [29]:
print(f"Accuracy: {accuracy_score(df_tweets_sampled.sentiment, df_tweets_sampled.prediction)}")

Accuracy: 0.8


### Structuring Unstructured Data

In [30]:
path = Path("./data/jobs")
filenames = [file.name for file in path.iterdir() if file.is_file()]
filenames = random.sample(filenames, 5)

jobs = []
for filename in filenames:
    file_path = path / filename
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
        jobs.append(content)

In [31]:
system_prompt = """Take the following job and extract data about the job. 

Respond with the following extracted data:
- job_title: The job title.
- job_class_no: The job class code.
- job_duties: The duties of the job.
- open_date: When the position was opened. Format: DD-MM-YYYY.
- salary: The salary ranges. Format: 'min salary to max salary'.
- deadline: The application deadline. Format: DD-MM-YYYY.
- application_form: The form of the application (e.g. online, fax, email).
- where_to_apply: The url to apply at or location to send the fax or email address."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class Job(BaseModel):
    job_title: str = Field(description="The job title.")
    job_class_no: int = Field(description="The job class code.")
    job_duties: str = Field(description="The duties of the job.")
    open_date: str = Field(description="When the position was opened. Format: DD-MM-YYYY.")
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")
    deadline: str = Field(description="The application deadline. Format: DD-MM-YYYY")
    application_form: str = Field(description="The form of the application (e.g. online, fax, email).")
    where_to_apply: str = Field(description="The url to apply at or location to send the fax or email address.")

job_data_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job}",
    task_prompt_variables=["job"],
    tools=None,
    output_format=Job,
    iterations=10,
)

In [32]:
def extract_jobs(jobs: list[str]) -> pd.DataFrame:
    data = []
    for job in jobs:
        response = job_data_agent.invoke({"job": job})
        data.append(response.final_answer)
    return pd.DataFrame(data)

In [33]:
df_jobs = extract_jobs(jobs)

11/05/24 12:02:48 INFO Prompt:
Job description:
GOLF STARTER
Class Code:       2453
Open Date:  12-11-15
(Exam Open to All, including Current City Employees)

ANNUAL SALARY

$42,428 to $57,169 

NOTE:

The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

A Golf Starter registers and schedules players at a City-owned golf course; receives refunds and accounts for revenue from fees, rentals and sales; operates a POS (Point of Sale) system; explains and enforces rules and regulations of the Department of Recreation and Parks; and patrols courses to expedite play.

REQUIREMENTS

1. One year of full-time paid experience with the City of Los Angeles with cash handling experience and direct public contact; or
2. Two years of full-time paid experience in golf course operations involving cash handling and direct public contact.

NOTES:

1. One year full-time experience is equivalent to 2,080 hours.
2

In [34]:
df_jobs.head()

,job_title,job_class_no,job_duties,open_date,salary,deadline,application_form,where_to_apply
0,GOLF STARTER,2453,A Golf Starter registers and schedules players...,12-11-2015,"[$42,428 to $57,169]",24-12-2015,online,http://agency.governmentjobs.com/lacity/defaul...
1,LAND SURVEYING ASSISTANT,7283,A Land Surveying Assistant sets up and operate...,12-08-2017,"[$66,440 to $97,133, $85,858 to $106,675]",21-12-2017,online,https://www.governmentjobs.com/careers/lacity
2,MARINE ENVIRONMENTAL MANAGER,9437,A Marine Environmental Manager directs or assi...,06-06-2014,"[$110,371 to $137,139, $122,690 to $152,444]",19-06-2014,online,http://agency.governmentjobs.com/lacity/defaul...
3,DIRECTOR OF PRINTING SERVICES,1488,The Director of Printing Services manages the ...,10-12-2018,"[$117,596 to $171,946]",01-11-2018,online,https://www.governmentjobs.com/careers/lacity/...
4,ENVIRONMENTAL SUPERVISOR,7304,"An Environmental Supervisor assigns, reviews a...",25-05-2018,"[$82,496 to $120,582, $89,637 to $131,063, $10...",07-06-2018,online,https://www.governmentjobs.com/careers/lacity


## 5. Multi-Agents

### Comparing Unstructured Data

In [35]:
def get_job(path: str) -> str:
    with open(path, "r", encoding="utf-8") as file:
        content = file.read()
        return content

job1 = get_job("./data/jobs/ELECTRICAL ENGINEERING ASSOCIATE 7525 093016 REV 100416.txt")
job2 = get_job("./data/jobs/ELECTRICAL MECHANIC 3841 012017.txt")

In [36]:
system_prompt = """Take the following job and extract data about the job. 

Respond with the following extracted data:
- job_title: The job title.
- job_duties: The duties of the job.
- salary: The salary ranges. Format: 'min salary to max salary'."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class Job1(BaseModel):
    job1_title: str = Field(description="The job title.")
    job1_duties: str = Field(description="The duties of the job.")
    salary1: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

class Job2(BaseModel):
    job2_title: str = Field(description="The job title.")
    job2_duties: str = Field(description="The duties of the job.")
    salary2: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

job_agent1 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job1}",
    task_prompt_variables=["job1"],
    tools=None,
    output_format=Job1,
    iterations=10,
)

job_agent2 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job2}",
    task_prompt_variables=["job2"],
    tools=None,
    output_format=Job2,
    iterations=10,
)

In [37]:
system_prompt = "Take the following 2 job descriptions and respond with the similarities and differences of the jobs."

task_prompt = """Compare the 2 given job descriptions:

Job 1:
Job title: {job1_title}
Job duties: 
{job1_duties}
Salary:
{salary1}


Job 2:
Job title: {job2_title}
Job duties: 
{job2_duties}
Salary:
{salary2}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class JobComparison(BaseModel):
    similarities: str = Field(description="The job similarities.")
    differences: str = Field(description="The job differences.")

job_comparison_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["job1_title", "job1_duties", "salary1", "job2_title", "job2_duties", "salary2"],
    tools=None,
    output_format=JobComparison,
    iterations=10,
)

In [38]:
chain = AgentChain(
    chain=[job_agent1, job_agent2, job_comparison_agent],
    chain_variables=["job1", "job2"],
)

In [39]:
response = chain.invoke({"job1": job1, "job2": job2})

11/05/24 12:04:12 INFO Prompt:
Job description:
ELECTRICAL ENGINEERING ASSOCIATE
Class Code:       7525
Open Date:  09-30-16
REVISED: 10-04-16
 (Exam Open to All, including Current City Employees)
ANNUAL SALARY 

$66,231 to $94,252; $74,082 to $105,444; $82,497 to $117,346; and $89,638 to $127,556
The salary in the Department of Water and Power is $77,360 to $96,110; $91,934 to $114,213; $99,722 to $123,881; and $107,156 to 
$133,130

NOTES:

1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.
2. For information regarding reciprocity between City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
3. The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

An Electrical Engineering Associate performs professional electrical engineering work in the preparation of designs, plans, specifications

In [40]:
print(response.final_answer["similarities"])

Both jobs are in the electrical field and involve working with electrical systems and equipment. They both may require working in various facilities and buildings.


In [41]:
print(response.final_answer["differences"])

The Electrical Engineering Associate is more focused on the design, planning, and quality assurance of electrical systems and equipment, and may also be involved in code enforcement. The Electrical Mechanic, on the other hand, is more involved in the hands-on installation and maintenance of electrical circuits and related equipment. The Electrical Engineering Associate has a wider and potentially higher salary range ($66,231 to $133,130) compared to the Electrical Mechanic ($69,655 to $99,514).


### Machine Learning Code Generation

In [42]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

task_prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description: 
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema: 
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class ModelingProblem(BaseModel):
    modeling_problem: str = Field(description="The machine learning problem.")

problem_finder_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_format=ModelingProblem,
    iterations=5,
)

In [43]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

task_prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema: 
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class AutoMLCode(BaseModel):
    code: str = Field(description="The Python machine learning code.")

ml_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_format=AutoMLCode,
    iterations=10,
)

In [44]:
ml_chain = AgentChain(
    chain=[problem_finder_agent, ml_agent],
    chain_variables=["problem_description", "dataset_size", "dataset_schema", "dataset_snippet"]
)

In [45]:
info_str = StringIO()
df_tweets.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [46]:
response = ml_chain.invoke({
    "problem_description": "I want to classify the sentiment of tweets.", 
    "dataset_size": len(df_tweets), 
    "dataset_schema": dataset_schema, 
    "dataset_snippet": str(df_tweets.head(10).to_markdown())
})

11/05/24 12:04:49 INFO Prompt:
Choose the machine learning problem best suited for the following problem and dataset.

Problem description: 
I want to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

11/05/24 12:04:54 INFO Raw response:
{
  "thought": "Given the problem description and the dataset, the user wants to classify the sentiment of tweets. This is a typical text classification problem, which falls under the category of supervised learning. Therefore, the best suited

In [47]:
print(response.final_answer["code"])

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Assuming df is the DataFrame
X = df['tweet']
y = df['sentiment']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the training data
X_train_vec = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_vec = vectorizer.transform(X_test)

# Initialize the MultinomialNB
clf = MultinomialNB()

# Fit the model
clf.fit(X_train_vec, y_train)

# Make predictions
y_pred = clf.predict(X_test_vec)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


### Forecasting

In [48]:
from datetime import timedelta
from language_models.tools.forecasting import get_earthquakes_data, ml_model

In [49]:
class Forecast(BaseModel):
    start_time: str = Field(None, description='Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')
    end_time: str = Field(None, description='Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')

def forecast(start_time = None, end_time = None):
    if start_time is None:
        start_time = (datetime.now() - timedelta(days=30)).date()
    if end_time is None:
        end_time = (datetime.now().date())
    df = get_earthquakes_data('https://earthquake.usgs.gov/fdsnws/event/1/query?', start_time, end_time)
    df_pred = ml_model.predict(df)
    return {'predictions': df_pred.to_dict(orient='records')}

In [50]:
forecasting_tool = Tool(func=forecast, name='forecast', description='Test forecast model on real-time events.', args_schema=Forecast)

In [51]:
task_prompt = """Take the following question and determine the start and end time to respond with.

Question:
{question}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=256,
    float=0.0,
)

class DateRange(BaseModel):
    start_time: str = Field(description="Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")
    end_time: str = Field(description="Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")

time_wizard_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt=task_prompt,
    task_prompt_variables=["question"],
    tools=[current_date_tool],
    output_format=DateRange,
    iterations=5,
)

In [52]:
forecast_chain = AgentChain(
    chain=[time_wizard_agent, forecasting_tool], 
    chain_variables=["question"],
)

In [53]:
response = forecast_chain.invoke({"question": "Run a forecast using the past week as data."})

11/05/24 12:05:15 INFO Prompt:
Take the following question and determine the start and end time to respond with.

Question:
Run a forecast using the past week as data.
11/05/24 12:05:18 INFO Raw response:
{
  "thought": "To answer this question, I need to determine the current date first.",
  "tool": "Current Date",
  "tool_input": {}
}
11/05/24 12:05:18 INFO Thought:
To answer this question, I need to determine the current date first.
11/05/24 12:05:18 INFO Tool:
Current Date
11/05/24 12:05:18 INFO Tool input:
{}
11/05/24 12:05:18 INFO Tool response:
2024-05-11 12:05:18.353736
11/05/24 12:05:32 INFO Raw response:
{
  "thought": "The current date is 2024-05-11. To run a forecast using the past week as data, the start time would be 7 days before the current date, and the end time would be the current date.",
  "tool": "Final Answer",
  "tool_input": {
    "start_time": "2024-05-04T12:05:18.353736",
    "end_time": "2024-05-11T12:05:18.353736"
  }
}
11/05/24 12:05:32 INFO Thought:
The cu

In [54]:
print(response.final_answer["forecast"])

{'predictions': [{'time': Timestamp('2024-05-04 12:09:23.240000+0000', tz='UTC'), 'prediction': 1.93495, 'latitude': 41.5813333333333, 'longitude': -112.417833333333, 'mag': 1.03, 'id': 'uu80068311', 'place': '16 km SW of Thatcher, Utah', 'location': '16 km SW of Thatcher, Utah'}, {'time': Timestamp('2024-05-04 12:17:24.816000+0000', tz='UTC'), 'prediction': 1.880359, 'latitude': 36.7705, 'longitude': -116.2558, 'mag': 0.4, 'id': 'nn00877176', 'place': '47 km ESE of Beatty, Nevada', 'location': '47 km ESE of Beatty, Nevada'}, {'time': Timestamp('2024-05-04 12:25:41.610000+0000', tz='UTC'), 'prediction': 1.956672, 'latitude': 33.5865, 'longitude': -116.8073333, 'mag': 0.16, 'id': 'ci40738368', 'place': '13 km WNW of Anza, CA', 'location': '13 km WNW of Anza, CA'}, {'time': Timestamp('2024-05-04 12:28:56.640000+0000', tz='UTC'), 'prediction': 2.266096, 'latitude': 45.842, 'longitude': -114.096666666667, 'mag': 1.09, 'id': 'mb90049753', 'place': '8 km W of Sula, Montana', 'location': '8 k

In [ ]:
app = FastAPI()

class Question(BaseModel):
    content: str

class Forecast(BaseModel):
    predictions: list[dict]

@app.get("/forecast")
def forecast(question: Question) -> Forecast:
    response = chain.invoke({"question": question.content})
    return response.final_answer["forecast"]

In [ ]:
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

### LLM-backed Tools

In [61]:
earthquake_agent.reset()
problem_finder_agent.reset()
ml_agent.reset()

In [62]:
class EarthquakeAgent(BaseModel):
    question: str = Field(description="The question regarding earthquakes.")

def answer_earthquake_questions(question: str) -> Any:
    response = earthquake_agent.invoke({"question": question})
    return response.final_answer

earthquake_agent_tool = Tool(
    func=answer_earthquake_questions,
    name="Earthquake Agent",
    description="Use this tool to answer questions about earthquakes.",
    args_schema=EarthquakeAgent,
)

class MLAgent(BaseModel):
    problem_description: str = Field(description="The user problem.")
    dataset_size: int = Field(description="The size of the dataset."), 
    dataset_schema: str = Field(description="The dataset schema or information."), 
    dataset_snippet: str = Field(description="The dataset snippet aka the first couple of rows of the dataset.")

def generate_ml_code(problem_description: str, dataset_size: int, dataset_schema: str, dataset_snippet: str) -> Any:
    response = ml_chain.invoke({
        "problem_description": problem_description,
        "dataset_size": dataset_size,
        "dataset_schema": dataset_schema,
        "dataset_snippet": dataset_snippet,
    })
    return response.final_answer

ml_agent_tool = Tool(
    func=generate_ml_code,
    name="ML Agent",
    description="Use this tool to generate machine learning code given a problem.",
    args_schema=MLAgent,
)

In [63]:
system_prompt = """You are an Agent that delegates tasks to other Agents by using the appropriate tools.

Use the Earthquake Agent when the question is about earthquakes.

Use the ML Agent when the user wants you to generate machine learning code."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=4096,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

almighty_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{prompt}",
    task_prompt_variables=["prompt"],
    tools=[earthquake_agent_tool, job_data_agent_tool, ml_agent_tool],
    output_format=Output,
    iterations=10,
)

In [64]:
response = almighty_agent.invoke({"prompt": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

11/05/24 12:08:59 INFO Prompt:
How many earthquakes have occurred for the past week with a magnitude of 5 or greater?
11/05/24 12:09:03 INFO Raw response:
{
  "thought": "The user is asking about earthquakes. I should use the Earthquake Agent to get this information.",
  "tool": "Earthquake Agent",
  "tool_input": {
    "question": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"
  }
}
11/05/24 12:09:03 INFO Thought:
The user is asking about earthquakes. I should use the Earthquake Agent to get this information.
11/05/24 12:09:03 INFO Tool:
Earthquake Agent
11/05/24 12:09:03 INFO Tool input:
{'question': 'How many earthquakes have occurred for the past week with a magnitude of 5 or greater?'}
11/05/24 12:09:03 INFO Prompt:
How many earthquakes have occurred for the past week with a magnitude of 5 or greater?
11/05/24 12:09:13 INFO Raw response:
{
  "thought": "First, I need to get the current date to calculate the start time for the past week.",


In [65]:
print(response.final_answer["content"])

There have been 30 earthquakes with a magnitude of 5 or greater in the past week.


In [66]:
prompt = f"""Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: {len(df_tweets)}
Schema: 
{dataset_schema}
First 10 rows of dataset:
{df_tweets.head(10).to_markdown()}"""

response = almighty_agent.invoke({"prompt": prompt})

11/05/24 12:09:27 INFO Prompt:
Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: 1600000
Schema: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

First 10 rows of dataset:
|    |   sentiment |         id | date                         | query    | user            | tweet                                                                                                               |
|---:|------------:|-----------:|:-----------------------------|:---------|:----------------|:----------------------------------

In [67]:
print(response.final_answer["content"])

Here is the Python code to train a model that predicts the sentiment of a tweet:

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Assuming the data is in a DataFrame df
# Let's take a smaller sample for initial model training
df_sample = df.sample(frac=0.1, random_state=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_sample['tweet'], df_sample['sentiment'], test_size=0.2, random_state=1)

# Convert the text data into a matrix of token counts
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_counts, y_train)

# Make predictions
y_pred = model.predict(X_test_